In [1]:
import tensorflow as tf
from tensorflow.python import keras
import tensorflow_graphics as tfg
import tensorflow_addons as tfa
import tensorflow_probability as tfp
from image_interpolation import ImageInterpolator, ImageSectionRNNCell, generate_2d_grid


2022-09-30 11:59:58.961210: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-30 11:59:59.196827: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-30 11:59:59.997599: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-09-30 11:59:59.997656: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
input=tf.keras.Input(shape=(28, 28,3))
num_iterations=20
repeated_input = tf.repeat(tf.expand_dims(input,1), repeats=num_iterations, axis=1)
print("repeated input: ",repeated_input)
classifier=keras.Sequential([
    tf.keras.layers.RNN(ImageSectionRNNCell(grid_dim=(8, 10), units=32)),
    tf.keras.layers.Dense(10, activation='softmax')
])
output=classifier(repeated_input)
model=tf.keras.Model(inputs=input, outputs=output)
model.summary()

2022-09-30 12:00:04.820895: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-30 12:00:04.820945: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ideapad5pro): /proc/driver/nvidia/version does not exist
2022-09-30 12:00:04.822160: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


repeated input:  KerasTensor(type_spec=TensorSpec(shape=(None, 20, 28, 28, 3), dtype=tf.float32, name=None), name='tf.repeat/Repeat/Reshape_1:0', description="created by layer 'tf.repeat'")
state_t:  (ListWrapper([<tf.Tensor 'module_wrapper/rnn/zeros:0' shape=(None, 32) dtype=float32>, <tf.Tensor 'module_wrapper/rnn/zeros_1:0' shape=(None, 32) dtype=float32>]), <tf.Tensor 'module_wrapper/rnn/zeros_2:0' shape=(None, 3) dtype=float32>)
image:  Tensor("module_wrapper/rnn/strided_slice_2:0", shape=(None, 28, 28, 3), dtype=float32)
starts Tensor("module_wrapper/rnn/image_section_rnn_cell/image_interpolator/strided_slice:0", shape=(None, 2), dtype=float32)
stops: Tensor("module_wrapper/rnn/image_section_rnn_cell/image_interpolator/add:0", shape=(None, 2), dtype=float32)
queries:  Tensor("module_wrapper/rnn/image_section_rnn_cell/image_interpolator/grid_generate/stack_2:0", shape=(None, 8, 10, 2), dtype=float32)
section_values:  Tensor("module_wrapper/rnn/image_section_rnn_cell/image_interpol

# Test differentiable grid generation

In [2]:
starts=tf.keras.Input(shape=(2,))
ends=tf.keras.Input(shape=(2,))
generate_2d_grid(starts, ends, (16, 20))

2022-09-30 11:22:11.191709: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-30 11:22:11.191793: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ideapad5pro): /proc/driver/nvidia/version does not exist
2022-09-30 11:22:11.193544: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<KerasTensor: shape=(None, 16, 20, 2) dtype=float32 (created by layer 'tf.stack_2')>

# Test differentiable batched interpolation

In [3]:
images= tf.keras.Input(shape=(28, 28, 3), name='images')
queries = tf.keras.Input(shape=(16, 16, 2), name='queries')
interpolated = tfp.math.batch_interp_regular_nd_grid(
            queries, [0.0, 0.0], [1.0, 1.0], images, axis=-3)
print(interpolated.shape)

(None, 16, 16, 3)


# Test that linspace is differentiable

In [8]:
inp=tf.keras.Input(shape=(2,))
interval=tf.linspace(inp[:,0],inp[:,1],10)
print(interval)
s=tf.reduce_sum(interval,axis=0)
print(s)
model=tf.keras.Model(inputs=inp,outputs=s)
#model.summary()
values=tf.Variable([[0.0,1.0],[1.0,2.0]])
with tf.GradientTape() as tape:
    tape.watch(values)
    out=model(values)
    print('output:',out)
    print(tape.gradient(out,values))

KerasTensor(type_spec=TensorSpec(shape=(None, None), dtype=tf.float32, name=None), name='tf.linspace_10/linspace/Slice:0', description="created by layer 'tf.linspace_10'")
KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name=None), name='tf.math.reduce_sum/Sum:0', description="created by layer 'tf.math.reduce_sum'")
array([[0., 1.],
       [1., 2.]], dtype=float32)>. This is a strong indication that the Lambda layer should be rewritten as a subclassed Layer.
array([[0., 1.],
       [1., 2.]], dtype=float32)>. This is a strong indication that the Lambda layer should be rewritten as a subclassed Layer.
output: tf.Tensor([ 5. 15.], shape=(2,), dtype=float32)
tf.Tensor(
[[5. 5.]
 [5. 5.]], shape=(2, 2), dtype=float32)


## Experiments with the grid interpolations

In [6]:
image = tf.constant(
    [[[[1.0], [2.0], [3.0]], [[4.0], [5.0], [6.0]], [[7.0], [8.0], [9.0]]]])
interpolator = ImageInterpolator(grid_dim=(3, 3))
print('Interpolation: ', interpolator(image, tf.constant([[0.0, 0.0, 0.5]])))
section = tf.Variable([[0.0, 0.0, 0.5]])
with tf.GradientTape() as tape:
    interpolation = interpolator(image, section)
    print('Interpolation: ', interpolation)
    # Take sum alon the last two axes
    s = tf.reduce_sum(interpolation, axis=1)
    s2 = tf.reduce_sum(s, axis=1)
    print("Sum: ", s2)
print("Gradient of sum w.r.t. section:", tape.gradient(s2, section))


image:  tf.Tensor(
[[[[1.]
   [2.]
   [3.]]

  [[4.]
   [5.]
   [6.]]

  [[7.]
   [8.]
   [9.]]]], shape=(1, 3, 3, 1), dtype=float32)
starts tf.Tensor([[0. 0.]], shape=(1, 2), dtype=float32)
stops: tf.Tensor([[0.5 0.5]], shape=(1, 2), dtype=float32)
queries:  tf.Tensor(
[[[[0.   0.  ]
   [0.   0.25]
   [0.   0.5 ]]

  [[0.25 0.  ]
   [0.25 0.25]
   [0.25 0.5 ]]

  [[0.5  0.  ]
   [0.5  0.25]
   [0.5  0.5 ]]]], shape=(1, 3, 3, 2), dtype=float32)
Interpolation:  tf.Tensor(
[[[[1. ]
   [1.5]
   [2. ]]

  [[2.5]
   [3. ]
   [3.5]]

  [[4. ]
   [4.5]
   [5. ]]]], shape=(1, 3, 3, 1), dtype=float32)
image:  tf.Tensor(
[[[[1.]
   [2.]
   [3.]]

  [[4.]
   [5.]
   [6.]]

  [[7.]
   [8.]
   [9.]]]], shape=(1, 3, 3, 1), dtype=float32)
starts tf.Tensor([[0. 0.]], shape=(1, 2), dtype=float32)
stops: tf.Tensor([[0.5 0.5]], shape=(1, 2), dtype=float32)
queries:  tf.Tensor(
[[[[0.   0.  ]
   [0.   0.25]
   [0.   0.5 ]]

  [[0.25 0.  ]
   [0.25 0.25]
   [0.25 0.5 ]]

  [[0.5  0.  ]
   [0.5  0.25]
   [0